In [ ]:
import boto3

In [ ]:
## Can we find the 'droids we're looking for?
def get_id_for_db(client,name):
    id = 'not-found'
    response = client.describe_db_instances()
    instances = response['DBInstances']
    for i in instances:
        if i['DBName'] == name.upper():
            id = i['DBInstanceIdentifier']
            break

    if id == 'not-found':
        raise ValueError('Database ' + db_name + ' not found in region ' + source_region)
    else:
        return id

In [ ]:
def get_source_snapshots(client,dbid):
    response = client.describe_db_snapshots(
         DBInstanceIdentifier=dbid
    )

    source_snapshots = []
    snapshots = response['DBSnapshots']
    for s in snapshots:
        if s['Status'] != 'available':
            continue
        source = {}
        source['id'] = s['DBSnapshotIdentifier']
        source['arn'] = s['DBSnapshotArn']
        source['state'] = s['Status']
        source_snapshots.append(source)

    return source_snapshots



In [ ]:
def get_dest_snapshots(client,dbid):
    response = client.describe_db_snapshots(
         DBInstanceIdentifier=dbid
    )

    dest_snapshots = []
    snapshots = response['DBSnapshots']
    for s in snapshots:
        dest_snapshots.append(s['DBSnapshotIdentifier'])

    return dest_snapshots



In [ ]:
def get_snaps_to_copy(src_snaps,dest_snaps):
    copy = []
    for ss in src_snaps:
        idMinusRDS = ss['id'].split(':')[1]
        if idMinusRDS not in dest_snaps:
            copy.append(ss)
            
    return copy
    




In [ ]:
targetParts = TargetDBSnapshotIdentifier=source_snapshots[0]['id'].split(':')
target = targetParts[1]
response = dest_client.copy_db_snapshot(
    SourceDBSnapshotIdentifier=source_snapshots[0]['arn'],
    TargetDBSnapshotIdentifier=target
)

print response

In [ ]:
## Setup

import boto3

source_region = 'eu-west-1'
dest_region = 'us-east-1'
db_name = 'dcdb'

source_client = boto3.client('rds',region_name=source_region)
dest_client = boto3.client('rds',region_name=dest_region)

id = get_id_for_db(source_client,db_name)
print id

snaps = get_source_snapshots(source_client,id)
print snaps

desc_snapshots = get_dest_snapshots(dest_client, id)
print desc_snapshots

s2c = get_snaps_to_copy(source_snapshots, dest_snapshots)

for s in s2c:
    target = s['id'].split(':')[1]
    print target

In [ ]:
## What about just listing the snapshots in a region?
import datetime

def get_older_snapshots(rds_client, threshold_hours):
    response = rds_client.describe_db_snapshots()
    snapshots = response['DBSnapshots']
    
    available = [s for s in snapshots if s['Status'] == 'available']
    
    print '**Original***'
    for s in available:
        print s['DBSnapshotIdentifier'],s['SnapshotCreateTime']
        
    print '**First 5'
    for s in available[:5]:
        print s['DBSnapshotIdentifier'],s['SnapshotCreateTime']
    
    available.sort(key=lambda item:item['SnapshotCreateTime'],reverse=False)
    print '**Sorted***'
    for s in available:
        print s['DBSnapshotIdentifier'],s['SnapshotCreateTime']
        
    print '**First 5'
    for s in available[:5]:
        print s['DBSnapshotIdentifier'],s['SnapshotCreateTime']
    
    #available = [s for s in snapshots if s['Status'] == 'available']
    tz_info = available[0]['SnapshotCreateTime'].tzinfo
    older = [s for s in available if s['SnapshotCreateTime']< (datetime.datetime.now(tz_info)-datetime.timedelta(hours=threshold_hours))]
    return older


In [ ]:
import boto3
rds_client = boto3.client('rds',region_name='us-east-1')
older_snapshots = get_older_snapshots(rds_client, 48)
for s in older_snapshots:
    print 'snapshot {} taken {}'.format(s['DBSnapshotIdentifier'], s['SnapshotCreateTime'])

In [ ]:
#Take a snap shop
import datetime

def timestamp():
    return datetime.datetime.utcnow().strftime("%Y-%m-%d-%H-%M")

def snapshot_it(rds_client, db_id):
    snapshot_id = 'ss-{}-{}'.format(
        db_id, str(timestamp())
    )
    print snapshot_id
    
    response = rds_client.create_db_snapshot(
        DBSnapshotIdentifier=snapshot_id,
        DBInstanceIdentifier=db_id
    )
    
    print response


In [ ]:
import boto3

rds_client = boto3.client('rds')
snapshot_it(rds_client,'dmbp41pr6lne05')

In [ ]:
import datetime
datetime.datetime.utcnow().strftime("%Y-%m-%d-%H-%M")


In [ ]:
ss = 'ss-dmbp41pr6lne05-1492526285777'
parts = ss.split(':')
if len(parts) == 1:
    print ss.split('-')[1]
else:
    print ss.split(':')[1]
    
print len(parts)
#'ss-dmbp41pr6lne05-1492526285777'.split(':')[1]

In [ ]:
# Scratch area

import boto3
import datetime

rds_client = boto3.client('rds', region_name='eu-west-1')
response = rds_client.describe_db_snapshots()
snapshots = response['DBSnapshots']
snapshots = [s for s in snapshots if s['SnapshotType'] == 'manual']
available = [s for s in snapshots if s['Status'] == 'available']
available.sort(key=lambda item:item['SnapshotCreateTime'])

tz_info = available[0]['SnapshotCreateTime'].tzinfo
older = [s for s in available if s['SnapshotCreateTime']< (datetime.datetime.now(tz_info)-datetime.timedelta(hours=36))]

for ss in available:
    print ss['DBSnapshotIdentifier']